# System Workflow

![Screenshot](./Snapshots/flow_diagram.png)

## Step-1 :
## Load the Youtube Transcripts based on TimeStamp Chunks

In [3]:
from langchain_community.document_loaders import YoutubeLoader # Load the Youtube Transcript
from langchain_community.document_loaders.youtube import TranscriptFormat # To Get transcripts as timestamped chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
from youtube_transcript_api import YouTubeTranscriptApi
from langchain_core.documents import Document
from collections import defaultdict
from datetime import timedelta
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
ytt_api = YouTubeTranscriptApi()
try:
    docs = ytt_api.fetch("K4CEsO9r1gU",languages=['en'])
except Exception as e:
    print("Transcript not found for this video .")

Transcript not found for this video .


## Step-2
## Loading the embedding model and the llm

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")
from langchain_huggingface import ChatHuggingFace,HuggingFaceEndpoint
from transformers import AutoTokenizer
# Initialize a llm model
# repo_id = "mistralai/Mistral-7B-Instruct-v0.3"
# # First load the tokenizer explicitly
# tokenizer = AutoTokenizer.from_pretrained(repo_id)
# llm1 = HuggingFaceEndpoint(
#     repo_id = repo_id,
#     temperature = 0.8,
#     max_new_tokens=500,
# )
# llm = ChatHuggingFace(llm=llm1,tokenizer=tokenizer)

In [ ]:
# !pip install langchain_groq

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name = "Qwen-Qwq-32b",max_tokens= 4000,model_kwargs={})

## Step-3
 
 ## Creating a vectordatabase using the Chroma db


In [ ]:
# vector_store = FAISS.from_documents(docs, embedding_model)
from langchain_chroma import Chroma
vectorstore = Chroma.from_documents(docs, embedding_model)

In [ ]:
# !pip install langchain-chroma
# !pip install lark

## Step-4 Defining the retriever
## Using the Metadatabased Filtering for retrievers

#### -> this retriever is known as self-query retriever

In [ ]:
print(docs[index].metadata)

In [ ]:
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The link of the video",
        type="string"
    ),
    AttributeInfo(
        name="start_seconds",
        description="The starting second of the video chunk (in seconds as integer)",
        type="integer"  # Changed from string to integer
    ),
    AttributeInfo(
        name="start_timestamp",
        description="Human-readable timestamp (HH:MM:SS format)",
        type="string"
    )
]

In [ ]:
# First get the base retriever from your vectorstore with increased k
base_vectorstore_retriever = vectorstore.as_retriever(
    # search_type = "mmr",
    search_kwargs={"k": 12,'lambda_mult':0.5}  # Increase this number as needed
)

In [ ]:
metadata_field_info

In [ ]:
document_content_description = "Transcript of a youtube video"
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
    # base_retriever = base_vectorstore_retriever,
    verbose=True,
    search_kwargs={"k": 12}  # Increase this number as needed

)

In [ ]:
# This example only specifies a filter
# retriever.invoke("Create me a blog post about the video.")
# retriever.invoke("what is meant by multi query retriever ?")

## Step- 5 Creating tools

### Tool A. VectorStore Retriever tool (Convert the rag_chain into a tool)
Redirect to this tool if the user queries is regarding the Video content

### Tool B. DuckDuckSeach Tool
Redirect to this tool if the user query is general

In [ ]:
from langchain_core.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun #Search user queries Online

@tool
def retriever_vectorstore_tool(query:str)->str:
    """Use this tool when the user ask about:
    - content of the youtube video
    - Any queries specifically about the youtube video 
    - If the user query involves providing summary , or about specific time stamp ,blog etc
    Input should be the exact search query.
    The tool will perform a vectorstore search using retriever."""
    return retriever.invoke(query)



search = DuckDuckGoSearchRun()
@tool
def duckducksearch_tool(query: str) -> str:
    """Use this tool Only when:
    - The question is about the current news, affairs etc.
    
    Input should be the exact search query.
    The tool will perform a web search using DuckDuckGo.
    """
    return search.invoke(query)

## Step-6 Binding the llm with the tools

In [ ]:
tools= [retriever_vectorstore_tool]
llm_with_tools=llm.bind_tools(tools=tools)

## Step-7 Define the langgraph workflow with memory

### Step-7.1 Define the State (flow of information through nodes)

In [ ]:
from typing_extensions import Annotated, TypedDict 
from typing import List 
from langchain_core.messages import AnyMessage,HumanMessage,SystemMessage #can be either HumanMsg or AImsg or ToolMsg
from langgraph.graph.message import add_messages #Append the new messages insted of replacing


class State(TypedDict):
    """Represents the state of our graph"""
    messages:Annotated[List[AnyMessage],add_messages]

### Step-7.2 Define the Graph

In [ ]:
from langgraph.graph import StateGraph, START, END
 # ToolNode is pre-built component that will invoke/execute the tool in behalf of the user and returns the tool_response
 # tools_condition is pre-built component that routes to ToolNode if the last message has tool call , otherwise routes to end
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display #to visualize the Graph
from langchain_core.messages import trim_messages # Trim the message and keep past 2 conversation
from langgraph.checkpoint.memory import MemorySaver #Implement langgraph memory


#### Step- 7.2.1 
#### Implement a ConversationalWindowBuffer Memory using langgraph

In [ ]:
# A function that determines the best tool to server the user query 
def tool_calling_llm(State:State)->State:
    selected_msg = trim_messages(
        State["messages"],
        token_counter=len,  # <-- len will simply count the number of messages rather than tokens
        max_tokens=10,  # <-- allow up to 10 messages (includes all AI ,human, tool msg : So have context about 2 previous conversations)
        strategy="last",
        start_on="human",
        # Usually, we want to keep the SystemMessage
        # if it's present in the original history.
        # The SystemMessage has special instructions for the model.
        include_system=True,
        allow_partial=False,
    )
    return {'messages':llm_with_tools.invoke(selected_msg)}

In [ ]:
tool_node = ToolNode(tools=tools)
tool_node

In [ ]:
#Initializing the StateGraph
builder = StateGraph(state_schema=State)


#Adding the nodes
builder.add_node('tool_calling_llm',tool_calling_llm) # returns the tools that is to be used
builder.add_node('tools',tool_node) # Executes the specified tool

#Adding Edges
builder.add_edge(START,'tool_calling_llm')
builder.add_conditional_edges(
    'tool_calling_llm',
    # If the latest message from AI is a tool call -> tools_condition routes to tools
    # If the latest message from AI is a not a tool call -> tools_condition routes to LLM, then generate final response and END
    tools_condition
)
builder.add_edge('tools','tool_calling_llm')

memory = MemorySaver()

#Compile the graph
graph = builder.compile(
    checkpointer=memory
)

# View
display(Image(graph.get_graph().draw_mermaid_png()))



### Code Explaination / Flow :

    1. Starts with calling the 'tool_calling_llm' , which decides which tool is to be used to answer the user query .

    2. It is redirected to the 'tools_condition' function , where 
        
        Case I: If the Last 'AI Message' is a tool call ,then 'tools_conditions' automatically routes to 'tool_node' which will executes the specified tool and return the tool_response.

        Case II: If the last 'AI Message' is not a tool call, then 'tools_conditions' routes to 'tool_calling_llm' generates the final reponse and route to END


In [ ]:
# Create a Unique Id for each user conversation
import uuid
thread_id = uuid.uuid4()
print(thread_id)
config = {"configurable": {"thread_id": thread_id}}

In [ ]:
query = "Explain in detail what is explained between minute 8 to 20 ?"
response = graph.invoke({
    'messages': query
},config=config)

In [ ]:
response

In [ ]:
for msg in response['messages']:
    msg.pretty_print()